# American market sales analysis

In [1]:
import pandas as pd

In [6]:
orders = pd.read_csv('orders.csv', index_col='id')
orders  # watch the table with orders

,order_date,ship_mode,customer_id,sales
id,,,,
100006,2014-09-07,Standard,DK-13375,377.970
100090,2014-07-08,Standard,EB-13705,699.192
100293,2014-03-14,Standard,NF-18475,91.056
100328,2014-01-28,Standard,JC-15340,3.928
100363,2014-04-08,Standard,JM-15655,21.376
...,...,...,...,...
168802,2017-11-03,Standard,JO-15145,18.368
169320,2017-07-23,Second,LH-16900,171.430
169488,2017-09-07,First,AA-10375,56.860


In [8]:
orders.dtypes  # clarify data type in columns

order_date      object
ship_mode       object
customer_id     object
sales          float64
dtype: object

In [10]:
round(orders.describe(), 2)  # all math data in sales column

,sales
count,5009.00
mean,458.61
std,954.73
min,0.56
25%,37.63
50%,151.96
75%,512.06
max,23661.23


### Check sales of each 10th customer

In [25]:
columns = ['customer_id', 'sales']
orders.iloc[::10][columns]

,customer_id,sales
id,,
100006,DK-13375,377.970
100867,EH-14125,321.552
101427,AY-10555,8.016
102274,DH-13075,865.500
103100,AB-10105,1107.660
...,...,...
157224,JB-16000,56.700
160465,SW-20350,466.814
163195,LL-16840,29.160


### Find sales > 1000 and shipped first class

In [27]:
sales_filter = 1000
ship_mode_filter = 'First'
orders.query('sales > @sales_filter & ship_mode == @ship_mode_filter')

,order_date,ship_mode,customer_id,sales
id,,,,
101931,2014-10-28,First,TS-21370,1252.602
103100,2014-12-20,First,AB-10105,1107.660
106726,2014-12-06,First,RS-19765,1261.330
112158,2014-12-02,First,DP-13165,1050.600
116666,2014-05-08,First,KT-16480,1799.970
...,...,...,...,...
147886,2017-03-28,First,DH-13075,1435.960
155425,2017-11-10,First,AB-10600,1475.054
162558,2017-10-02,First,Dp-13240,2437.672


### How much money the store earned with each delivery class

In [29]:
pd.options.display.float_format = '{:,.1f}'.format
orders.groupby('ship_mode')['sales'].sum()

ship_mode
First        351,428.4
Same Day     128,363.1
Second       459,193.6
Standard   1,358,215.7
Name: sales, dtype: float64

### Find 10 best sales days

In [30]:
orders.groupby(['ship_mode', 'order_date'])['sales'].agg(['sum']).sort_values(by='sum', ascending=False).head(10)

sum
ship_mode order_date         
Standard  2014-03-18 26,908.4
          2016-10-02 18,398.2
First     2017-03-23 14,299.1
Standard  2014-09-08 14,060.4
First     2017-10-22 13,716.5
Standard  2016-12-17 12,185.1
          2017-11-17 12,112.5
          2015-09-17 11,467.6
          2016-05-23 10,561.0
          2014-09-23 10,478.6

## Let's merge 2 dataframes

In [34]:
customers = pd.read_csv('customers.csv', index_col='id')
customers # watch the table with customers

,name,segment,state,city
id,,,,
CG-12520,Claire Gute,Consumer,Kentucky,Henderson
DV-13045,Darrin Van Huff,Corporate,California,Los Angeles
SO-20335,Sean O'Donnell,Consumer,Florida,Fort Lauderdale
BH-11710,Brosina Hoffman,Consumer,California,Los Angeles
AA-10480,Andrew Allen,Consumer,North Carolina,Concord
...,...,...,...,...
CJ-11875,Carl Jackson,Corporate,Pennsylvania,Philadelphia
RS-19870,Roy Skaria,Home Office,Iowa,Burlington
SC-20845,Sung Chung,Consumer,Illinois,Arlington Heights


### Check all orders from city Kent

In [37]:
city_filter = 'Kent'
customers.query('city == @city_filter')

,name,segment,state,city
id,,,,
JC-15385,Jenna Caffey,Consumer,Washington,Kent
JH-15820,John Huston,Consumer,Ohio,Kent
RE-19405,Ricardo Emerson,Consumer,Ohio,Kent


### Clients from what state buy the most

In [40]:
# First we need to merge two dataframes.
new_df = pd.merge(orders, customers, how='inner', left_on='customer_id', right_index=True)
new_df

,order_date,ship_mode,customer_id,sales,name,segment,state,city
id,,,,,,,,
100006,2014-09-07,Standard,DK-13375,378.0,Dennis Kane,Consumer,Ohio,Marion
131884,2015-12-06,Same Day,DK-13375,594.0,Dennis Kane,Consumer,Ohio,Marion
145065,2015-12-12,First,DK-13375,32.3,Dennis Kane,Consumer,Ohio,Marion
133046,2017-07-27,Second,DK-13375,298.0,Dennis Kane,Consumer,Ohio,Marion
165099,2017-12-11,First,DK-13375,1.4,Dennis Kane,Consumer,Ohio,Marion
...,...,...,...,...,...,...,...,...
168193,2017-03-06,Second,RM-19750,98.3,Roland Murray,Consumer,New York,New York City
122140,2015-04-02,Standard,MO-17950,130.3,Michael Oakman,Consumer,Texas,Dallas
166233,2017-07-03,Standard,MO-17950,24.0,Michael Oakman,Consumer,Texas,Dallas


In [78]:
state_df = new_df.groupby('state')['sales'].agg(['sum'])  # Make sum for all states
state_df.sort_values(by='sum', ascending=False).head(1)   # Find best state for sales

,sum
state,
California,"451,036.6"


### Find 5 cities that have best sales in 2016

In [80]:
# First find orders in 2016
orders_2016 = orders.query("order_date >= '2016-01-01' & order_date <= '2016-12-31'")
orders_2016

,order_date,ship_mode,customer_id,sales
id,,,,
100041,2016-11-20,Standard,BF-10975,328.5
100083,2016-11-24,Standard,CD-11980,24.8
100153,2016-12-13,Standard,KH-16630,63.9
100244,2016-09-20,Standard,GM-14695,475.7
100300,2016-06-24,Second,MJ-17740,"4,823.1"
...,...,...,...,...
168620,2016-12-24,Second,RB-19795,823.6
169040,2016-12-06,Standard,GT-14710,"1,169.5"
169369,2016-12-12,Standard,EC-14050,299.1


In [81]:
# City is in customers dataframe, so we need to merge two dataframes.
with_customers_2016 = pd.merge(customers, orders_2016, how='inner', left_index=True, right_on='customer_id')

In [84]:
# Let's group by cities and calculate sales.
grouped_2016 = with_customers_2016.groupby('city')['sales'].sum()
grouped_2016.head()

city
Akron               1,763.0
Albuquerque           692.9
Amarillo              197.2
Arlington           5,672.1
Arlington Heights      14.1
Name: sales, dtype: float64

In [85]:
# Let's sort and get top-5.
top5 = grouped_2016.sort_values(ascending=False).head(5)
print(top5)

city
New York City   53,094.1
Philadelphia    39,895.5
Seattle         33,955.5
Los Angeles     33,611.1
San Francisco   27,990.0
Name: sales, dtype: float64


### How much orders were shipped by First class in 2017

In [89]:
orders_2017 = orders.query("order_date >= '2017-01-01' & order_date <= '2017-31-12'")
orders_2017

,order_date,ship_mode,customer_id,sales
id,,,,
100013,2017-11-06,Standard,ZC-21910,29.7
100055,2017-05-28,Standard,MD-17860,152.6
100097,2017-11-26,Second,MN-17935,"1,115.2"
100111,2017-09-20,Standard,SV-20365,"7,359.9"
100160,2017-09-07,Standard,CB-12025,49.7
...,...,...,...,...
168802,2017-11-03,Standard,JO-15145,18.4
169320,2017-07-23,Second,LH-16900,171.4
169488,2017-09-07,First,AA-10375,56.9


In [107]:
orders_2017_FC = orders_2017.query("ship_mode == 'First'")
orders_2017_FC.count()
# Answer is 284

order_date     284
ship_mode      284
customer_id    284
sales          284
dtype: int64

### How much clients from California

In [117]:
customers_California = customers.query("state == 'California'")
customers_California.describe()
# Answer is 161 (unique)

,name,segment,state,city
count,161,161,161,161
unique,161,3,1,37
top,Frank Gastineau,Consumer,California,Los Angeles
freq,1,87,161,58


### How much orders do clients from California

In [118]:
customers_California

,name,segment,state,city
id,,,,
DV-13045,Darrin Van Huff,Corporate,California,Los Angeles
BH-11710,Brosina Hoffman,Consumer,California,Los Angeles
ZD-21925,Zuschuss Donatelli,Consumer,California,San Francisco
EH-13945,Eric Hoffmann,Consumer,California,Los Angeles
RA-19885,Ruben Ausman,Corporate,California,Los Angeles
...,...,...,...,...
DO-13645,Doug O'Connell,Consumer,California,Los Angeles
AO-10810,Anthony O'Donnell,Corporate,California,Los Angeles
VP-21760,Victoria Pisteka,Corporate,California,San Francisco


In [122]:
orders_California = pd.merge(orders, customers_California, how='inner', left_on='customer_id', right_index=True)
orders_California

,order_date,ship_mode,customer_id,sales,name,segment,state,city
id,,,,,,,,
100328,2014-01-28,Standard,JC-15340,3.9,Jasper Cacioppo,Consumer,California,Los Angeles
102792,2016-12-13,Standard,JC-15340,9.8,Jasper Cacioppo,Consumer,California,Los Angeles
154536,2016-09-05,Standard,JC-15340,9.6,Jasper Cacioppo,Consumer,California,Los Angeles
129441,2017-09-07,Standard,JC-15340,47.9,Jasper Cacioppo,Consumer,California,Los Angeles
100678,2014-04-18,Standard,KM-16720,697.1,Kunst Miller,Consumer,California,Los Angeles
...,...,...,...,...,...,...,...,...
162978,2017-05-04,Standard,LW-16990,502.5,Lindsay Williams,Corporate,California,San Francisco
102239,2016-05-05,First,LW-16990,"1,691.6",Lindsay Williams,Corporate,California,San Francisco
118542,2017-12-01,Standard,CC-12550,55.5,Clay Cheatham,Consumer,California,San Francisco


In [123]:
orders_California.describe(include='all')
# Answer is 1006 sales orders.

,order_date,ship_mode,customer_id,sales,name,segment,state,city
count,1006,1006,1006,"1,006.0",1006,1006,1006,1006
unique,645,4,161,nan,161,3,1,37
top,2015-11-02,Standard,PG-18820,nan,Sally Hughsby,Consumer,California,Los Angeles
freq,6,587,13,nan,13,565,1006,357
mean,NaN,NaN,NaN,448.3,NaN,NaN,NaN,NaN
std,NaN,NaN,NaN,748.5,NaN,NaN,NaN,NaN
min,NaN,NaN,NaN,0.9,NaN,NaN,NaN,NaN
25%,NaN,NaN,NaN,43.1,NaN,NaN,NaN,NaN
50%,NaN,NaN,NaN,168.4,NaN,NaN,NaN,NaN
75%,NaN,NaN,NaN,518.8,NaN,NaN,NaN,NaN


### Make a pivot table of average sales for all states for each ship mode

In [125]:
orders_merged = pd.merge(orders, customers, how='inner', left_on='customer_id', right_index=True)
orders_merged

,order_date,ship_mode,customer_id,sales,name,segment,state,city
id,,,,,,,,
100006,2014-09-07,Standard,DK-13375,378.0,Dennis Kane,Consumer,Ohio,Marion
131884,2015-12-06,Same Day,DK-13375,594.0,Dennis Kane,Consumer,Ohio,Marion
145065,2015-12-12,First,DK-13375,32.3,Dennis Kane,Consumer,Ohio,Marion
133046,2017-07-27,Second,DK-13375,298.0,Dennis Kane,Consumer,Ohio,Marion
165099,2017-12-11,First,DK-13375,1.4,Dennis Kane,Consumer,Ohio,Marion
...,...,...,...,...,...,...,...,...
168193,2017-03-06,Second,RM-19750,98.3,Roland Murray,Consumer,New York,New York City
122140,2015-04-02,Standard,MO-17950,130.3,Michael Oakman,Consumer,Texas,Dallas
166233,2017-07-03,Standard,MO-17950,24.0,Michael Oakman,Consumer,Texas,Dallas


In [127]:
state_pt = pd.pivot_table(orders_merged, values='sales', index='state', columns='ship_mode', aggfunc='mean')
state_pt

ship_mode,First,Same Day,Second,Standard
state,,,,
Alabama,537.0,265.6,796.9,442.1
Arizona,458.7,413.6,776.8,619.5
Arkansas,nan,7.8,42.3,561.3
California,396.0,488.2,419.7,468.7
Colorado,430.7,283.2,398.5,436.4
Connecticut,334.5,"1,179.3","1,230.7",341.4
Delaware,174.8,89.4,467.3,659.7
District of Columbia,697.3,647.8,nan,68.5
Florida,201.1,568.6,313.1,386.7


### Get total sales for each state and for each year

In [133]:
orders_merged["date"] = pd.to_datetime(orders_merged['order_date'])
orders_merged.head()

,order_date,ship_mode,customer_id,sales,name,segment,state,city,date
id,,,,,,,,,
100006,2014-09-07,Standard,DK-13375,378.0,Dennis Kane,Consumer,Ohio,Marion,2014-09-07
131884,2015-12-06,Same Day,DK-13375,594.0,Dennis Kane,Consumer,Ohio,Marion,2015-12-06
145065,2015-12-12,First,DK-13375,32.3,Dennis Kane,Consumer,Ohio,Marion,2015-12-12
133046,2017-07-27,Second,DK-13375,298.0,Dennis Kane,Consumer,Ohio,Marion,2017-07-27
165099,2017-12-11,First,DK-13375,1.4,Dennis Kane,Consumer,Ohio,Marion,2017-12-11


In [134]:
orders_merged.set_index('date').resample('Y')["sales"].sum()

date
2014-12-31   484,247.5
2015-12-31   470,532.5
2016-12-31   609,205.6
2017-12-31   733,215.3
Freq: A-DEC, Name: sales, dtype: float64

In [135]:
orders_merged.set_index('date').groupby('state')["sales"].resample("Y").sum() # Variant 1

state       date      
Alabama     2014-12-31    3,829.7
            2015-12-31    6,016.4
            2016-12-31   13,669.6
            2017-12-31    7,523.3
Arizona     2014-12-31   11,403.9
                           ...   
Washington  2017-12-31   47,843.9
Wisconsin   2014-12-31    1,908.6
            2015-12-31    9,974.4
            2016-12-31    7,555.7
            2017-12-31   21,531.5
Name: sales, Length: 162, dtype: float64

In [136]:
orders_merged.groupby(['state', pd.Grouper(key='date', freq='Y')])['sales'].sum() # Variant 2

state       date      
Alabama     2014-12-31    3,829.7
            2015-12-31    6,016.4
            2016-12-31   13,669.6
            2017-12-31    7,523.3
Arizona     2014-12-31   11,403.9
                           ...   
Washington  2017-12-31   47,843.9
Wisconsin   2014-12-31    1,908.6
            2015-12-31    9,974.4
            2016-12-31    7,555.7
            2017-12-31   21,531.5
Name: sales, Length: 161, dtype: float64